<a href="https://colab.research.google.com/github/Kai1219/colab-python-data-practice/blob/main/News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 新聞語料關鍵字分析 jieba

In [1]:
import requests
from bs4 import BeautifulSoup

# 利用 requests 對 API 來源發送一個請求
response = requests.get('https://udn.com/news/breaknews/1')
soup = BeautifulSoup(response.text)

news = []

for link in soup.find_all('h3', class_='rounded-thumb__title')[:4]:
  news_url = link.a['href']
  news_response = requests.get('https://udn.com' + news_url)
  news_soup = BeautifulSoup(news_response.text)
  news_content = news_soup.find('div', class_='article-content__paragraph').text.strip().replace('\n', ' ')
  news.append(news_content)

print(news)

['8月7日是節氣「立秋」，為陽氣漸消、陰氣漸長的轉捩點，而因正值閏月雙春年，當天又巧遇天赦日，民俗專家廖大乙說，立秋前後3天理髮可改運，也提醒龍、狗、牛、羊4生肖及農曆歲數逢9男性、逢6的女性要「躲秋」可化煞納福。 \r 廖大乙指出，24節氣中的「立秋」代表即將告別炎熱的夏天，迎來秋高氣爽的日子，同時是陽氣漸消、陰氣漸長的一年中的轉捩點，而今年立秋不僅落在閏六月，還巧遇今年第4個天赦日，磁場尤為特殊，是化煞納福及改運、轉運的大好機會。                      \r 倘若覺得自身上半年運勢欠佳，除了立秋逢天赦日當天在見天處，雙手合十誠心向玉皇大帝懺悔求特赦開恩，或分送雞、鴨蛋等帶殼食物，象徵由剝而復、否極泰來，更建議要把握立秋的前後3天去剪頭髮，改變造型讓自己從「頭」開始。 \r 此外，想改運除了捐血、拜廟之外，最近中南部災情嚴重，民眾不妨及時行善布施，能捐助金錢物資給災區或協助災民重建家園，而在人手一機時代，多分享正面、勸善或勵志的好文章，帶給周邊或網路上的人滿滿正能量也是一種布施。 \r 廖大乙也提醒，由於今年的立秋較為特殊，可視為運勢變換的重要轉折點，因此化煞納福改運的效果更佳，建議去年犯太歲的龍、狗、牛、羊4生肖，以及農曆歲數逢9的男性、逢6的女性等6種人在立秋當天要記得「躲秋」為宜。 \r 而龍、狗、牛、羊4生肖，以及農曆歲數逢9的男性、逢6的女性，若在立秋當天午時，也就是上午11時至下午1時，盡量待在室內或不見天的地方，若因故不得不出門，那就戴頂帽子再外出，成功趨吉避凶後，下半年運勢可望一飛沖天。   ☛此為民俗說法，不代表本新聞網立場，切勿過度迷信。  民俗專家廖大乙提醒，閏月立秋逢天赦日，龍、狗、牛、羊4生肖及農曆歲數逢9男性、逢6的女性要「躲秋」可化煞納福。圖／廖大乙提供圖為入秋後常見的甜根子草。本報資料照     立秋 生肖 天赦日', '台積電內部爆出疑技術外洩事件，知情人士指出，部分前員工在任職期間，違規取得2奈米晶片製程的關鍵技術，涉案人員已遭解職。台灣高檢署指出，檢方已於7月25至28日陸續傳喚、拘提涉案3名工程師，訊後向智財及商業法院聲請羈押禁見獲准。 \r 據了解，檢調有搜索一間名為Tokyo Electron「日本東京威力」廠商，該廠商在新竹科學園區設有辦公室。台積電2奈米製程開發難度高，投入成本極高，目前全球僅有

In [4]:
# 安裝 jieba 中文斷詞套件
!pip install jieba

# 下載官方字典檔
!wget https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big


# 載入套件與字典檔
import jieba
jieba.load_userdict("dict.txt.big")

--2025-08-05 13:44:15--  https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8583143 (8.2M) [application/octet-stream]
Saving to: ‘dict.txt.big’

dict.txt.big        100%[===================>]   8.18M  --.-KB/s    in 0.09s   

2025-08-05 13:44:16 (86.3 MB/s) - ‘dict.txt.big’ saved [8583143/8583143]



Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.828 seconds.
DEBUG:jieba:Loading model cost 0.828 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [5]:
# 將每篇新聞文章斷詞後的結果存入 `tokens` 變數，並且將斷詞後的結果計算成每個單字的出現次數（詞頻），
# 並存成以「出現單字為 KEY、出現次數為 Value」的 dict 型態變數 `word_count`。

# 精確模式斷詞

tokens = []  # 儲存所有斷詞結果

for d in news:
  token = list(jieba.cut(d, HMM=False)) # token = d 字串斷詞結果，會輸出陣列
  tokens.extend(token)

word_count = {}

# 將斷詞後的結果計算成每個單字的出現次數（詞頻）
for w in tokens:
  if w not in word_count:
    word_count[w] = 0
  word_count[w] += 1

word_count = sorted(word_count.items(),key=lambda x:x[1],reverse=True)
print(word_count)
# {'聲明': 4, '主管機關': 3, ...}

[(' ', 211), ('，', 128), ('。', 41), ('台', 41), ('的', 40), ('、', 39), ('積', 37), ('電', 30), ('奈', 21), ('米', 21), ('為', 20), ('\r', 19), ('在', 19), ('2', 19), ('「', 16), ('」', 16), ('！', 16), ('秋', 14), ('及', 12), ('員工', 12), ('製程', 12), ('📢', 12), ('立', 11), ('日', 10), ('也', 10), ('逢', 10), ('取得', 10), ('調查', 10), ('是', 9), ('6', 9), ('技術', 9), ('已', 9), ('最', 9), ('天', 8), ('等', 8), ('後', 8), ('資料', 8), ('晶片', 8), ('工程師', 8), ('先進', 8), ('4', 7), ('指出', 7), ('外', 7), ('洩', 7), ('遭', 7), ('全球', 7), ('與', 7), ('並', 7), ('》', 7), ('大', 6), ('中', 6), ('或', 6), ('人', 6), ('前', 6), ('目前', 6), ('被', 6), ('營業', 6), ('公司', 6), ('發現', 6), ('看', 6), ('赦', 5), ('廖', 5), ('乙', 5), ('3', 5), ('生肖', 5), ('1', 5), ('關鍵技術', 5), ('涉案人員', 5), ('高檢', 5), ('署', 5), ('涉案', 5), ('見', 5), ('有', 5), ('研發', 5), ('國家', 5), ('秘密', 5), ('法律', 5), ('這', 5), ('可能', 5), ('新聞', 5), ('外商', 5), ('人員', 5), ('漸', 4), ('而', 4), ('當天', 4), ('改', 4), ('運', 4), ('龍', 4), ('狗', 4), ('牛', 4), ('羊', 4), ('農曆', 4), ('歲數', 4), ('